<a href="https://colab.research.google.com/github/PSN-01/EPA-per-Target-vs.-Targets-Top-75-Pass-Catchers-/blob/main/pa_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nfl_data_py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 47.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
xarray 2025.1.2 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.1 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 24.12.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.


In [ ]:
import nfl_data_py as nfl
import pandas as pd

df = nfl.import_pbp_data([2024])
teams = nfl.import_team_desc()[["team_abbr", "team_color", "team_color2"]]

df_receivers = df[(df["pass_attempt"] == 1) & (df["receiver_player_name"].notna())]

latest_team = df_receivers.groupby("receiver_player_name")["posteam"].last().reset_index()

receivers_impact = df_receivers.groupby("receiver_player_name").agg(
    total_epa=("epa", "sum"),
    targets=("receiver_player_name", "count")
).reset_index()


receivers_impact["epa_per_target"] = receivers_impact["total_epa"] / receivers_impact["targets"]

receivers_impact = receivers_impact.merge(latest_team, on="receiver_player_name", how="left")

receivers_impact = receivers_impact.merge(teams, left_on="posteam", right_on="team_abbr", how="left")

top_100_receivers = receivers_impact.sort_values(by="total_epa", ascending=False).head(100)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

top_100_receivers.to_excel("top_100_receivers.xlsx", index=False)

print(top_100_receivers)